In [ ]:
!pip install -q transformers datasets evaluate

In [ ]:
!pip install -U datasets huggingface_hub fsspec

  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 has no pad_token by default

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))  # Resize in case pad_token was added

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Embedding(50257, 768)

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=50,
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/tmp/ipython-input-10-41011983.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ganesh-sonawane-sarc (ganesh-sonawane-sarc-iit-bombay) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,3.183800,3.310683
2,3.017900,3.302290


TrainOutput(global_step=18360, training_loss=3.220186026371642, metrics={'train_runtime': 3474.8511, 'train_samples_per_second': 21.134, 'train_steps_per_second': 5.284, 'total_flos': 4797060415488000.0, 'train_loss': 3.220186026371642, 'epoch': 2.0})

In [ ]:
output_dir = "/content/drive/MyDrive/Colab Notebooks/Colab works/NLP LS/next word resutls"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('/content/drive/MyDrive/Colab Notebooks/Colab works/NLP LS/next word resutls/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/Colab works/NLP LS/next word resutls/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/Colab works/NLP LS/next word resutls/vocab.json',
 '/content/drive/MyDrive/Colab Notebooks/Colab works/NLP LS/next word resutls/merges.txt',
 '/content/drive/MyDrive/Colab Notebooks/Colab works/NLP LS/next word resutls/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/Colab works/NLP LS/next word resutls/tokenizer.json')

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 27.17


In [ ]:
import torch

def compute_top_k_accuracy(dataset, model, tokenizer, k=3, max_samples=1000):
    model.eval()
    correct = 0
    total = 0

    for i, example in enumerate(dataset.select(range(max_samples))):
        input_ids = torch.tensor(example['input_ids']).unsqueeze(0).to(model.device)

        with torch.no_grad():
            outputs = model(input_ids)
            logits = outputs.logits  # shape: [1, seq_len, vocab_size]

        # Skip the first token (no context to predict it)
        for j in range(1, input_ids.size(1)):
            true_token = input_ids[0, j]
            pred_logits = logits[0, j - 1]  # Predict token j

            # Skip padding/eos
            if true_token.item() in [tokenizer.pad_token_id, tokenizer.eos_token_id]:
                continue

            top_k = pred_logits.topk(k).indices
            if true_token in top_k:
                correct += 1
            total += 1

    return correct / total if total > 0 else 0
top_k_acc = compute_top_k_accuracy(tokenized_datasets['validation'], model, tokenizer)
print(f"Top-3 Accuracy: {top_k_acc:.2%}")

Top-3 Accuracy: 56.97%


In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
generator("The future of Machine Learning is", max_new_tokens=20)

Device set to use cuda:0


[{'generated_text': 'The future of Machine Learning is unclear. Machine learning has proven to be a critical component in the development of computer vision and artificial intelligence'}]

In [ ]:
import pandas as pd

log_history = trainer.state.log_history
df = pd.DataFrame(log_history)
df.to_csv(f"{output_dir}/training_log.csv", index=False)